In [1]:
from pyspark.sql import SparkSession
from datetime import datetime, timedelta

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
8,application_1603482505045_0655,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# Define date
year = "2020"
month = "06"
day = "03"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# Global variables
database = "covid_project"
etl1_table = "{}.etl1_ntbk_aggregated".format(database)
etl2_table = "{}.covid_twitter_etl2_ntbk".format(database)
output_path = "s3://<bucket-name>/covid_etl3/"
output_table = "{}.covid_etl3_ntbk_v2".format(database)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# Process data from yesterday (Tuesday) to last Wednesday
dt = "{}-{}-{}".format(year, month, day)
this_day = datetime.strptime(dt, "%Y-%m-%d")
tues = this_day - timedelta(days=1)
last_wed = this_day - timedelta(days=7)

yesterday = tues.strftime("%Y-%m-%d")
that_day = last_wed.strftime("%Y-%m-%d")
processed_days = []

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# Function to generate proper WHERE clause to query 7 days worth of data from ETL1 and ETL2 output. This logic may not be required if running every day.
def get_where_clause():
    # Prepare a WHERE clause for ETL1 data based on month and day values for last week
    m_d = {}
    for i in range(7):
        d = tues - timedelta(days=i)
        # Add day to processed_days list
        processed_days.append({'S': d.strftime("%Y-%m-%d")})

        m = '{}-{}'.format(d.year, "{0:0=2d}".format(d.month))
        if m not in m_d: m_d[m] = ["{0:0=2d}".format(d.day)]
        else: m_d[m].append("{0:0=2d}".format(d.day))

    conditions = []
    for m, d in m_d.items():
        conditions.append('month = "{}" AND day IN ("{}")'.format(m, '", "'.join(d)))

    return '({})'.format(') OR ('.join(conditions))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# Create Session
spark = SparkSession \
    .builder \
    .appName("ETL3 - Join Github and Twitter data") \
    .enableHiveSupport() \
    .config("spark.hadoop.mapreduce.input.fileinputformat.input.dir.recursive","true") \
    .config("hive.mapred.supports.subdirectories","true") \
    .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# Create a Dataframe from ETL-2 results (Twitter data)
q1 = 'select country, cast(day as date), numberofcovidtweetsretrieved as numberofcovidtweetsretrieved from {} where country != "" AND cast(day as date) between cast("{}" as date) and cast("{}" as date)'.format(etl2_table, that_day, yesterday)
df1 = spark.sql(q1)
df1.show(4)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+----------------------------+
|country|       day|numberofcovidtweetsretrieved|
+-------+----------+----------------------------+
|     PK|2020-05-29|                          72|
|     AU|2020-05-29|                          75|
|     SA|2020-05-29|                          17|
|     CA|2020-05-29|                         211|
+-------+----------+----------------------------+
only showing top 4 rows

In [14]:
# Create a Dataframe from ETL-1 results (Github data)
q2 = 'select country, cast(concat(month, "-", day) as date) as dt, sum(confirmed) as confirmed, sum(deaths) as deaths, sum(recovered) as recovered from {} where {} group by country, dt'.format(etl1_table, get_where_clause())
df2 = spark.sql(q2)
df2.show(4)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+---------+------+---------+
|country|        dt|confirmed|deaths|recovered|
+-------+----------+---------+------+---------+
|     ET|2020-05-29|      968|     8|      197|
|     BE|2020-05-29|    58061|  9430|    15682|
|     SZ|2020-05-29|      279|     2|      168|
|     SA|2020-05-29|    81766|   458|    57013|
+-------+----------+---------+------+---------+
only showing top 4 rows

In [15]:
# Join above two dataframes 
df3 = df2.join(df1, (df1.country == df2.country) & (df1.day == df2.dt)).select(df1.country, df1.day, df1.numberofcovidtweetsretrieved, df2.confirmed, df2.deaths, df2.recovered)
df3.show(4)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+----------------------------+---------+------+---------+
|country|       day|numberofcovidtweetsretrieved|confirmed|deaths|recovered|
+-------+----------+----------------------------+---------+------+---------+
|     ET|2020-05-29|                           4|      968|     8|      197|
|     BE|2020-05-29|                          19|    58061|  9430|    15682|
|     SZ|2020-05-29|                           4|      279|     2|      168|
|     SA|2020-05-29|                          17|    81766|   458|    57013|
+-------+----------+----------------------------+---------+------+---------+
only showing top 4 rows

In [16]:
df3.repartition(1).write.mode("append").option("path", output_path).format("Parquet").saveAsTable(output_table)
print("Wrote the output!")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Wrote the output!